In [ ]:
import pymongo

username = "student"
password = "student"
host = "10.20.66.7:27017"
database_name = "fsig-raw"

url = f"mongodb://{username}:{password}@{host}/{database_name}"

try:
    # Utwórz klienta MongoDB
    myclient = pymongo.MongoClient(url)

    # Uzyskanie dostępu do bazy danych
    mydb = myclient[database_name]

    # Sprawdzenie dostępności bazy danych
    dblist = myclient.list_database_names()
    if database_name in dblist:
        print("Connected to MongoDB successfully.")
        print("Available databases:", dblist)
    else:
        print("Database not found.")

except Exception as e:
    print("An error occurred:", e)
